In [1]:
#import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#define model, train function, evaluate function

def build_model(input_shape, output_shape):
    model = models.Sequential([
    tf.keras.layers.Input(input_shape),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(output_shape)
])
    return model

def train_model(model, train_data, train_labels, epochs=10, batch_size=32):
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size)
    return history

def evaluate_model(model, test_data, test_labels):
    loss = model.evaluate(test_data, test_labels)
    return loss

In [3]:
#load data 
#separate into 9 datasets for 9 invariants assoicated with each permutation

filepath = '' #for data

perm_enc = []  

data_list1 = []
data_list2 = []
data_list3 = []
data_list4 = []
data_list5 = []
data_list6 = []
data_list7 = []
data_list8 = []
data_list9 = []

with open(filepath, 'r') as f:
    datalist = []
    for line in f.readlines()[1:]:
        datalist.append([list(t) for t in eval(line)])

len(datalist[0][1][0]) #check number of coefficients for each invariant is 256

for i in range(len(datalist)):
    perm= datalist[i][0]

    num_elements = len(perm)
    one_hot_perm = np.zeros((num_elements, num_elements))
    for j, elem in enumerate(perm):
        one_hot_perm[j, elem] = 1
    perm_enc.append(one_hot_perm.reshape(64,))

    coeff_list_i = datalist[i][1]
    data_list1.append(coeff_list_i[0])
    data_list2.append(coeff_list_i[1])
    data_list3.append(coeff_list_i[2])
    data_list4.append(coeff_list_i[3])
    data_list5.append(coeff_list_i[4])
    data_list6.append(coeff_list_i[5])
    data_list7.append(coeff_list_i[6])
    data_list8.append(coeff_list_i[7])
    data_list9.append(coeff_list_i[8])

data_list_9 = [data_list1,data_list2,data_list3,data_list4,data_list5,data_list6,data_list7,data_list8,data_list9]

#collect subinvariants for each invariant i
collected = []

for list in data_list_9:
    coeffall = []
    coeff0 = []
    coeff2 = []
    coeff4 = []
    coeff6 = []
    coeff8 = []
    for i in range(len(perm_enc)):
        coeffall.append(list[i])
        coeff0.append([list[i][0]])
        coeff2.append(list[i][9:37])
        coeff4.append(list[i][93:163])
        coeff6.append(list[i][219:247])
        coeff8.append([list[i][255]])
    collected.append([coeffall,coeff0,coeff2,coeff4,coeff6,coeff8])


select = [1,2,3,4,5,4,3,2,1] #only use non-zero subinvariants

data_size = len(perm_enc)

In [4]:
#lists to store final accuracies and gradients
finalacc = []
finalgradient = []

In [ ]:
#regression: input permutation, output invariant/subinvariant, for each invariant 0-8

for i in range(0,9): 
    print(f'Invariant: {i}')  #inv i

    allcoeff = collected[i]

    k = 5 
    d = -2

    finalaccfori = [] #collect the acc for full inv + subinvariants
    #errors = []

    #grad_all = []
    avegradfori = [] #collect the mean grad for full inv + subinvariants

    #for full invariant and each non-zero subinvariant
    for coeff in allcoeff[:select[i]+1]: 

        print(f'{d}-vector') # subinv d, use -2 for the full invariant

        # ave over 100 rounds of training

        grad_all = []
        acc_all = []

        for loop in range(100): 

            ML_data = [[perm_enc[index],coeff[index]] for index in range(data_size)]
            
            np.random.shuffle(ML_data) 

            tl = int(0.8 * data_size)- int(0.8 * data_size) % k
            dataset, final_test_dataset = ML_data[:tl], ML_data[tl:]
            s = int(np.floor(len(dataset)/k))

            Training_inputs, Training_outputs, Testing_inputs, Testing_outputs = [], [], [], []
            for j in range(k):
                Training_inputs.append([datapoint[0] for datapoint in dataset[:j*s]]+[datapoint[0] for datapoint in dataset[(j+1)*s:]])
                Training_outputs.append([datapoint[1] for datapoint in dataset[:j*s]]+[datapoint[1] for datapoint in dataset[(j+1)*s:]])
                Testing_inputs.append([datapoint[0] for datapoint in dataset[j*s:(j+1)*s]])
                Testing_outputs.append([datapoint[1] for datapoint in dataset[j*s:(j+1)*s]])
                
        
            input_shape = (64,)
            output_shape = len(coeff[0])

            model = build_model(input_shape, output_shape)

            testloss = []

            for fold in range(k):
                print(f'Fold {fold+1}/{k}')

                history = train_model(model, np.array(Training_inputs)[fold], np.array(Training_outputs)[fold], epochs=10, batch_size=32)
                print('...trained')

                test_loss = evaluate_model(model, np.array(Testing_inputs)[fold], np.array(Testing_outputs)[fold])
                testloss.append(test_loss)
                print(f'Test Loss: {test_loss:.4f}')
            
            #print(f'Invariant: {i}, {d}-vector, Test Loss: {np.mean(testloss):.4f} ± {np.std(testloss):.4f}')
            
            #test with unseen data
            finalTesting_inputs, finalTesting_outputs = [], []

            for datapoint in final_test_dataset:
                finalTesting_outputs.append(datapoint[1])
                finalTesting_inputs.append(datapoint[0])

            finaltest_loss = evaluate_model(model,np.array(finalTesting_inputs), np.array(finalTesting_outputs))
            print(f'Invariant: {i}, {d}-vector, Final Test Loss: {finaltest_loss:.4f}')   

            #calculate the accuracy after rounding the prediction to nearest integer
            Pred = model.predict(np.array(finalTesting_inputs))
            count = []
            roundPred = []
            correct = []
            result = []
            errorposition = []
            for m in range(len(np.array(finalTesting_outputs))):
                rounded_list = [round(elem) for elem in Pred[m] ]
                #rounded_list = [round(elem*2)/2 for elem in Pred[m]]
                roundPred.append(np.array(rounded_list))
                if rounded_list == np.array(finalTesting_outputs[m]).tolist():
                    count.append(1)
                else:
                    positionenc = finalTesting_inputs[m].reshape(8,8)
                    position = []
                    for row in positionenc:
                        for l in range(8):
                            if row[l] == 1:
                                position.append(l)
                    errorposition.append(position)
                res = np.array(rounded_list)==np.array(finalTesting_outputs[m])
                cor = sum(np.array(rounded_list)==np.array(finalTesting_outputs[m]))
                correct.append(cor)
                result.append(res)
            accs1 = np.sum(count)/len(np.array(finalTesting_outputs))
            #accs2 = np.sum(correct)/len(np.concatenate(finalTesting_outputs))
            
            acc_all.append(accs1)
            #errors.append(errorposition)

            #gradient saliency for each of the 100 loops
            image = tf.Variable(finalTesting_inputs)
            with tf.GradientTape(watch_accessed_variables=False) as tape:
                tape.watch(image)
                predictions = model(image)
                loss = predictions
            
            gradient = tape.gradient(loss, image)
            gradient = gradient.numpy()
            avg_grad = np.absolute(np.mean(gradient,axis=0)).reshape(1,64)
            avg_grad_8 = []
            for r in range(1,9):
                a = np.mean(np.absolute(np.mean(gradient,axis=0))[8*(r-1):8*r])
                avg_grad_8.append(a)
            avg_grad_8 = np.array(avg_grad_8).reshape(1,8)
            grad_all.append(avg_grad_8)

            
        #calculate the average gradients and accuracies
        avegrad = np.mean(grad_all,axis = 0) 
        avegradfori.append(avegrad)
        plt.axis('off')
        plt.imshow(avegrad,aspect='auto')
        plt.savefig('D8_'+str(i)+str(d)+'_ave.png',bbox_inches='tight')

        finalaccfori.append(np.mean(acc_all))

        d = d+2 
        

    finalgradient.append(avegradfori)
    finalacc.append(finalaccfori)
